In [2]:
import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import eigsh


## Method 1: Determining Curvature from Point Cloud Distribution using normals from neighboring points

In [26]:
pcd = o3d.io.read_point_cloud("./point_cloud_data/coupler/injection-molded-coupler.ply")
pcd = pcd.voxel_down_sample(voxel_size=0.005)
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(
    radius=0.02, max_nn=50))
X = np.asarray(pcd.points)

determine the curvature using the local surface variation

In [27]:

nn = 30
nbrs = NearestNeighbors(n_neighbors=nn, algorithm='kd_tree').fit(X)
idx = nbrs.kneighbors(return_distance=False)
curv = np.empty(len(X))
for i, N in enumerate(idx):
    P = X[N] - X[i]
    C = (P.T @ P) / (len(N)) 
    w = np.linalg.eigvalsh(C)        # λ1<=λ2<=λ3
    curv[i] = w[0] / (w.sum() + 1e-12)

In [28]:
k = 12
nbrs = NearestNeighbors(n_neighbors=k, algorithm='kd_tree').fit(X)
dist, inds = nbrs.kneighbors(X)
sigma2 = np.median(dist[:,1:]**2)  
rows, cols, vals = [], [], []
for i in range(len(X)):
    for d,j in zip(dist[i,1:], inds[i,1:]): 
        w = np.exp(-d*d / (sigma2+1e-12))
        rows += [i, j]; cols += [j, i]; vals += [w, w]
W = csr_matrix((vals,(rows,cols)), shape=(len(X),len(X)))
D = diags(np.ravel(W.sum(axis=1)))
L = D - W       # calculate the graph laplacian
# eigen-embeddings (lowest nontrivial eigenvectors)
m = 8
evals, evecs = eigsh(L.asfptype(), k=m+1, M=D.asfptype(), sigma=0.0)
spec = evecs[:,1:m+1]

group the features to run kmeans on them

In [29]:
N = np.asarray(pcd.normals)
feat = np.c_[curv, N, spec] # group the features
feat = (feat - feat.mean(0)) / (feat.std(0)+1e-12)

k_segments = 3
labels = KMeans(n_clusters=k_segments, n_init=10, random_state=0).fit_predict(feat)

3d visualizer

In [30]:
colors = np.random.RandomState(42).rand(k_segments,3)
pcd.colors = o3d.utility.Vector3dVector(colors[labels])
o3d.visualization.draw_geometries([pcd])